In [ ]:
from datetime import datetime, timedelta
import json
from pytz import timezone as tz
import requests


In [ ]:
def argsFunc(*args, **kwargs):
    return args, kwargs


result = argsFunc("arg1", "arg2", "arg3", key1="value1", key2="value2")

args, kwargs = result
for arg in args:
    print(f"Positional argument: {arg}")

for key, value in kwargs.items():
    print(f"Keyword argument: {key} = {value}")


Positional argument: arg1
Positional argument: arg2
Positional argument: arg3
Keyword argument: key1 = value1
Keyword argument: key2 = value2


In [ ]:
print(CURR_SEASON := datetime.now().year)
print(type(CURR_SEASON))


2026
<class 'int'>


In [ ]:
EPList: list = [
	{
		"endpoint": "seasons",
		"url": BASE_URL + "/seasons/all?sportId=1",
		"pathParamUrl": BASE_URL + "/seasons/?sportId=1{**kwargs}",
	},
	{
		"endpoint": "leagues",
		"url": BASE_URL + "/leagues?sportId=1",
		"pathParamUrl": BASE_URL + "/leagues?sportId=1&leagueIds={leagueIds}",
		"pathParamUrl": BASE_URL + "/leagues/{leagueId}",
	},
]


In [ ]:
def printUrl(endpoint: str, **kwargs) -> None:
    baseUrl = "https://statsapi.mlb.com/api/v1"
    url = f"{baseUrl}/{endpoint}?sportId=1"
    for k, v in kwargs.items():
        url += f"&{k}={v}"
    print(url)


printUrl("leagues", leagueIds="103")


https://statsapi.mlb.com/api/v1/leagues?sportId=1&leagueIds=103


In [ ]:
# gamePks = 746406,746400
baseUrl = "https://statsapi.mlb.com/api/v1"
print(
    json.dumps(
        requests.get(f"{baseUrl}/leagues?sportId=1&leagueIds=103,104").json(),
        indent=4,
    )
)


{
    "copyright": "Copyright 2026 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt",
    "leagues": [
        {
            "id": 103,
            "name": "American League",
            "link": "/api/v1/league/103",
            "abbreviation": "AL",
            "nameShort": "American",
            "seasonState": "preseason",
            "hasWildCard": true,
            "hasSplitSeason": false,
            "numGames": 162,
            "hasPlayoffPoints": false,
            "numTeams": 15,
            "numWildcardTeams": 3,
            "seasonDateInfo": {
                "seasonId": "2026",
                "preSeasonStartDate": "2026-01-01",
                "preSeasonEndDate": "2026-02-19",
                "seasonStartDate": "2026-02-20",
                "springStartDate": "2026-02-20",
                "springEndDate": "2026-03-24",
                "regularSeasonStartDate": "2026-03-25",

In [177]:
_type = "scheduleTypes"
typeUrl = f"https://statsapi.mlb.com/api/v1/{_type}"

_typeResponse = requests.get(typeUrl)
_typeData = _typeResponse.json()
# print(json.dumps(data, indent=4))
if isinstance(_typeData, list):
    for item in _typeData:
        itemContent = [f"{key}: {value}" for key, value in item.items()]
        print(", ".join(itemContent))
else:
    print(json.dumps(_typeData, indent=4))


id: games, description: game schedule
id: xref, description: xref games
id: events, description: all events schedule


In [ ]:
url = "https://statsapi.mlb.com/api/v1/schedule?sportId=1"

# url = "https://statsapi.mlb.com/api/v1/jobs?jobType=HeadCoach"
# url = "https://statsapi.mlb.com/api/v1/game/745604/playByPlay"
response = requests.get(url)
data = response.json()
print(json.dumps(data, indent=4))


{
    "error": "Not Found",
    "path": "api/v1/schedules",
    "status": 404,
    "timestamp": "2026-01-28T21:43:46.486579006Z"
}


In [2]:
def renderUrl(
    ep: str = None, epExt: list = None, pParams: dict = None, **kwargs
) -> str:
    url: str = f"https://statsapi.mlb.com/api/v1{f'/{ep}' if ep else ''}"
    url += f"/{pParams.get(list(pParams.keys())[0])}" if pParams else ""
    url += f"/{'/'.join(ext for ext in epExt)}" if epExt else ""
    url += (
        f"/{'/'.join(str(v) for k, v in list(pParams.items())[1:])}"
        if len(list(pParams.keys())) > 1
        else ""
    )
    url += f"?{'&'.join(f'{k}={v}' for k, v in kwargs.items())}" if kwargs else ""

    return url


sampleUrl1: str = renderUrl(
    ep="teams", epExt=["roster"], pParams={"teamId": 144}, hydrate="person"
)
sampleUrl2: str = renderUrl(
    ep="people", epExt=["stats", "game"], pParams={"playerId": 660670, "gameId": 747124}
)

print(sampleUrl1)
print(sampleUrl2)


def buildUrl(endpoint: str = "", pathParams: dict = None, **kwargs) -> str:
    baseUrl: str = "https://statsapi.mlb.com/api/v1"
    endpoints: list | None = (
        list(map(str.strip, endpoint.split(","))) if endpoint else None
    )
    mainEndpoint: str = f"/{endpoints[0]}" if endpoints else ""
    endpointBranches: str = (
        f"/{'/'.join(endpoints[1:])}" if endpoints and len(endpoints) > 1 else ""
    )
    pathParam1: str = (
        f"/{pathParams.get(list(pathParams.keys())[0])}" if pathParams else ""
    )
    pathParam2: str = (
        f"/{pathParams.get(list(pathParams.keys())[1])}"
        if pathParams and len(list(pathParams.keys())) > 1
        else ""
    )
    queryParams: str = (
        f"?{'&'.join(f'{k}={v}' for k, v in kwargs.items())}" if kwargs else ""
    )

    return f"{baseUrl}{mainEndpoint}{pathParam1}{endpointBranches}{pathParam2}{queryParams}"


sampleUrl3: str = buildUrl("teams, roster", {"teamId": 144}, hydrate="person")
sampleUrl4: str = buildUrl(
    "people, stats, game", {"playerId": 660670, "gameId": 747124}
)

print(sampleUrl3)
print(sampleUrl4)


https://statsapi.mlb.com/api/v1/teams/144/roster?hydrate=person
https://statsapi.mlb.com/api/v1/people/660670/stats/game/747124
https://statsapi.mlb.com/api/v1/teams/144/roster?hydrate=person
https://statsapi.mlb.com/api/v1/people/660670/stats/game/747124


In [ ]:
def callAPI(endpoint: str = "", pathParams: dict = None, **kwargs) -> dict:
    url: str = buildUrl(endpoint, pathParams, **kwargs)
    response: requests.Response = requests.get(url)
    data: dict = json.dumps(response.json(), indent=2)
    return data


In [4]:
sampleCall1: dict = callAPI("teams, roster", {"teamId": 144}, hydrate="person")

print(sampleCall1)


{
  "copyright": "Copyright 2026 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt",
  "roster": [
    {
      "person": {
        "id": 700363,
        "fullName": "AJ Smith-Shawver",
        "link": "/api/v1/people/700363",
        "firstName": "AJ",
        "lastName": "Smith-Shawver",
        "primaryNumber": "32",
        "birthDate": "2002-11-20",
        "currentAge": 23,
        "birthCity": "Fort Worth",
        "birthStateProvince": "TX",
        "birthCountry": "USA",
        "height": "6' 3\"",
        "weight": 205,
        "active": true,
        "primaryPosition": {
          "code": "1",
          "name": "Pitcher",
          "type": "Pitcher",
          "abbreviation": "P"
        },
        "useName": "AJ",
        "useLastName": "Smith-Shawver",
        "boxscoreName": "Smith-Shawver",
        "gender": "M",
        "isPlayer": true,
        "isVerified": true,
       

In [5]:
sampleCall2: dict = callAPI(
    "people, stats, game", {"playerId": 660670, "gameId": 747124}
)

print(sampleCall2)


{
  "copyright": "Copyright 2026 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt",
  "stats": [
    {
      "exemptions": [],
      "splits": [
        {
          "stat": {
            "gamesStarted": 1,
            "caughtStealing": 0,
            "stolenBases": 0,
            "stolenBasePercentage": ".---",
            "caughtStealingPercentage": ".---",
            "assists": 1,
            "putOuts": 1,
            "errors": 0,
            "chances": 2,
            "fielding": ".000",
            "passedBall": 0,
            "pickoffs": 0
          },
          "type": "gameLog",
          "group": "fielding"
        },
        {
          "stat": {},
          "type": "gameLog",
          "group": "pitching"
        },
        {
          "stat": {
            "summary": "0-5 | R",
            "gamesPlayed": 1,
            "flyOuts": 2,
            "groundOuts": 3,
            "a

In [6]:
def getLogoUrl(id: int, type: str = "cap", shade: str = "light") -> str:
    baseUrl = "https://www.mlbstatic.com/team-logos"
    return f"{baseUrl}/team-{type}-on-{shade}/{id}.svg"


print(getLogoUrl(144))


https://www.mlbstatic.com/team-logos/team-cap-on-light/144.svg
